In [3]:
import torch
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

квантование модели - техника уменьшения её размера и как следствие ускорения скорости инференса. Достигается это с помощью  
изменения типа тензоров весов и/или активаций. Как известно, веса нейросети это типы данных с плавающей точкой. Квантование  
же переводит их в инты. Достигается это, очевидно, домножением на некоторый скейл фактор (читай скаляр). Стратегия выбора  
такого скаляра определяет различные способоы квантования. Однако в торче есть динамическое квантование, которое использует  
не заранее заданный скаляр, а определяет его на основании значений данных, полученный в ходе рантайма. Попробуем разобраться  
с этим.

Для подготовим нейросеть, которую будем квантовать. В качестве датасета возьмем FashionMNIST

In [59]:
import torch.nn as nn 

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 6,kernel_size=5,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten()
        )

        self.fc1 = nn.Sequential(
           
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.ReLU(inplace=True),

            nn.Linear(120,84),
            nn.ReLU(inplace=True),

            nn.Linear(84,10)
        )
    
    def forward(self,x):
        out = self.cnn1(x)
        #out = out.view(out.size()[0], -1)
        out = self.fc1(out)
        return out
    

In [60]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from torch.utils.data import DataLoader

trainingData = CIFAR10(
    root = "./data/CIFAR10/",
    train = True,
    download=False,
    transform = T.ToTensor()
)
testData = CIFAR10(
    root = "./data/CIFAR10/",
    train = False,
    download = False,
    transform = T.ToTensor()
)



In [169]:
import copy
def train(model, dl, lossFunc, optimizer, device):
    size = len(dl.dataset)
    for batch, (X,y) in enumerate(dl):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X)
        loss = lossFunc(out, y)
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def validate(model, dl, lossFunc, device, toPrint=True):
    size = len(dl.dataset)
    numBatches = len(dl)
    testLoss, correct = 0, 0
    model = model.to('cpu')
    with torch.no_grad():
        for X, y in dl:
            X, y = X.to("cpu"), y.to("cpu")
            pred = model(X)
            testLoss += lossFunc(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    testLoss /= numBatches
    correct /= size
    if toPrint:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")
    model = model.to(device)
    return 100*correct

def loop(model, traindl, testdl, optimizer, lossFunc, epochs, device):
    bestAcc = 0.0
    bestEp = 0
    best = None
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")
        model.train()
        train(model, traindl,lossFunc,optimizer, device)
        model.eval()
        acc = validate(model, testdl, lossFunc, device)
        if acc > bestAcc:
            bestAcc = acc
            best = copy.deepcopy(model.state_dict())
            bestEp = bestEp
    return best, bestAcc


In [70]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
trainDL = DataLoader(trainingData, shuffle=True, batch_size=64)
testDL = DataLoader(testData, shuffle=True, batch_size=64)
model = SimpleCNN()
lossF = CrossEntropyLoss()
optim = SGD(model.parameters(), lr=0.003, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
print(device)
model = model.to(device)


cuda:0


In [ ]:
loop(model, trainDL, testDL, optim, lossF, epochs, device)

In [ ]:
#torch.save(model.state_dict(), './models/best.pth')
trainDL = DataLoader(trainingData, shuffle=True, batch_size=64)
testDL = DataLoader(testData, shuffle=True, batch_size=64)

model = SimpleCNN()
model.load_state_dict(torch.load("./models/best.pth"))
model = model.to(device)
optim = torch.optim.Adam(model.parameters(), 0.0001)
bestmodel, bestacc = loop(model, trainDL, testDL, optim, lossF, epochs, device)

наверное 64% это максимум, который я получу

Ладно вещи оказались намного сложнее, чем я думал. Сейчас со всем разберемся

In [158]:
from torch.quantization import QuantStub, DeQuantStub

class HUI(nn.Module):
    def __init__(self):
        super(HUI,self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 6,kernel_size=5,stride=1),
            nn.ReLU(inplace=True)            
        )
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.cnn2 = nn.Sequential(
            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size=5, stride=1),
            nn.ReLU(inplace=True)
        )
        self.maxpool2 = self.maxpool1
        self.flatten = nn.Flatten()

        self.fc1 = nn.Sequential(
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.ReLU(inplace=True)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(inplace=True)
        )
        self.fc3 = nn.Linear(84,10)
        
    
    def forward(self,x):
        
        #out = self.quant(x)
        out = self.cnn1(x)        
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.maxpool2(out)
        out = self.flatten(out)
        
        out = self.fc1(out)
        
        out = self.fc2(out)
        out = self.fc3(out)
        #out = self.dequant(out) 
        
        return out
        
    def fuse(self):
        torch.quantization.fuse_modules(self.cnn1, ['0','1'], inplace= True)
        torch.quantization.fuse_modules(self.cnn2, ['0','1'], inplace= True)
        torch.quantization.fuse_modules(self.fc1, ['0','1'], inplace= True)
        torch.quantization.fuse_modules(self.fc2, ['0','1'], inplace= True)
        
        


#### Post training Static Quantization

PTQ static квантует веса и активации уже после тренировки. Для того, чтобы вычислить верный 
scale фактор, ему требует калибровка на соотвествующем датасете. Помимо этого нужно выполнить  
еще не сколько условий:
1. необходимо указать где квантовать активации. Для этого используется (De)QuantStub
2. зафьюзить слои. fuse_modules. Это нужно, чтобы во время калиброки функции активации и  
   слои, предшествующие им были квантовались вместе.  

PTQ static используется, когда важно сэкономить как объем памяти, так и время вычислений, что  
как раз свойственно CNN

In [94]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "./models/temp.p")
    print('Size (MB):', os.path.getsize("./models/temp.p")/1e6)
    os.remove('./models/temp.p')

Модифицируем нашу нейросеть. Добавляем Квантование активаций и фьюзим слои, которые  
торч умеем фьюзить 

In [167]:
class staticlyQuantizedCNN(nn.Module):
    def __init__(self,nonQuantizedModel):
        super(staticlyQuantizedCNN, self).__init__()
        self.nonQuantizedModel = nonQuantizedModel
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
    
    def forward(self, x):
        x = self.quant(x)
        x = self.nonQuantizedModel(x)
        x = self.dequant(x)
        return x

def fuseSQCNN(model:staticlyQuantizedCNN ) -> None:
    torch.quantization.fuse_modules(model.nonQuantizedModel.cnn1, ['0','1'], inplace=True)
    torch.quantization.fuse_modules(model.nonQuantizedModel.cnn1, ['3','4'], inplace=True)
    torch.quantization.fuse_modules(model.nonQuantizedModel.fc1, ['0','1'], inplace=True)
    torch.quantization.fuse_modules(model.nonQuantizedModel.fc1, ['2','3'], inplace=True)



In [171]:
bestModel = SimpleCNN()
bestModel.load_state_dict(torch.load("./models/best.pth")) 
bestModel = bestModel.cpu()
bestModel.eval()
model = staticlyQuantizedCNN(bestModel)
model.eval()
model = model.cpu()

print ("original model'size")
print_size_of_model(model)

fuseSQCNN(model)
model.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model, inplace=True)
print("original model'accuracy")
validate(model, testDL, lossF, "cpu")
torch.quantization.convert(model, inplace=True)
print("fused model's accuracy")
validate(model, testDL, lossF, "cpu")
print("fused model's size")
print_size_of_model(model)

bestModel = SimpleCNN()
bestModel.load_state_dict(torch.load("./models/best.pth")) 
bestModel.eval()
bestModel = bestModel.cpu()
bestModel.eval()
model = staticlyQuantizedCNN(bestModel)
model.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model, inplace=True)
validate(model, testDL, lossF, "cpu", toPrint=False)
torch.quantization.convert(model, inplace=True)
print("nonfused quantizedModel model'accuracy")
validate(model, testDL, lossF, "cpu")
print("nonfused quantizedModel model'size")
print_size_of_model(model)



original model'size
Size (MB): 0.251591
original model'accuracy
Test Error: 
 Accuracy: 63.4%, Avg loss: 1.222992 

fused model's accuracy
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.233569 

fused model's size
Size (MB): 0.070243
nonfused quantizedModel model'accuracy
Test Error: 
 Accuracy: 62.7%, Avg loss: 1.248018 

nonfused quantizedModel model'size
Size (MB): 0.070243


Наглядная демонстрация почему необходимо фьюзить слои. Квантованная модель, у которой слои не были  
слеплены вместе показывает просадку по точности, большую, чем та, где соотвествующие слои были слеплены  
Отличия  в нашем случае не драматические, вероятнее всего из-за малого размера исходной нейросети. Хотя,  
признаться, сжатие в размерах произошло значительное.

#### Quantization Aware Training for Static Quantization

Quantization Aware Training - как следует из названия, этот метод предполагает моделирование  
поведения квантования во время обучения, что дает большую точность по сравнению с другими методами.  
Во время обучения все вычисления осуществляются с помощью чисел с плавающей точкой, но при этом  
к ним добавляются fake_quants, чтобы симулировать поведение int8. После обучения, модель квантуется 
по правилам PTQ Static

перепишем нашу нейронку 

In [187]:
class QAwareSimpleCNN(nn.Module):
    def __init__(self):
        super(QAwareSimpleCNN,self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 6,kernel_size=5,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten()
        )

        self.fc1 = nn.Sequential(
           
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.ReLU(inplace=True),

            nn.Linear(120,84),
            nn.ReLU(inplace=True),

            nn.Linear(84,10)
        )
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
    
    def forward(self,x):
        out = self.quant(x)
        out = self.cnn1(out)
        out = self.fc1(out)
        out = self.dequant(out)
        return out
    
    def fuse(self):
        torch.quantization.fuse_modules(self.cnn1, ['0','1'], inplace=True)
        torch.quantization.fuse_modules(self.cnn1, ['3','4'], inplace=True)
        torch.quantization.fuse_modules(self.fc1, ['0','1'], inplace=True)
        torch.quantization.fuse_modules(self.fc1, ['2','3'], inplace=True)

In [188]:
model = QAwareSimpleCNN()
model.train()
model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model.fuse()
preparedModel = torch.quantization.prepare_qat(model)

d:\torch\torch\lib\site-packages\torch\ao\quantization\observer.py:177: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [189]:
trainDL = DataLoader(trainingData, shuffle=True, batch_size=64)
testDL = DataLoader(testData, shuffle=True, batch_size=64)
lossF = CrossEntropyLoss()
optim = torch.optim.Adam(preparedModel.parameters(), 0.0001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 20
print(device)
preparedModel = preparedModel.to(device)

cuda:0


In [ ]:
model, acc = loop(preparedModel,trainDL, testDL, optim, lossF, epochs, device)

In [ ]:
optim = torch.optim.Adam(preparedModel.parameters(), 0.0001)
model, acc = loop(preparedModel,trainDL, testDL, optim, lossF, epochs, device)

In [199]:
validate(preparedModel,testDL, lossF, device)

Test Error: 
 Accuracy: 57.1%, Avg loss: 1.193975 



57.099999999999994

In [197]:
preparedModel.eval()
preparedModel = preparedModel.cpu()
modelInt8 = torch.quantization.convert(preparedModel)
print_size_of_model(preparedModel)
print_size_of_model(modelInt8)

Size (MB): 0.280032
Size (MB): 0.075653
